In [1]:
import tensorflow as tf
import datetime
import os
import numpy as np
import ctypes
from PIL import Image

In [2]:
#### ------------------------------------------- ####
#### ------------- LOAD THE DATA --------------- ####
#### ------------------------------------------- ####

In [3]:
MAIN_DATASET_FOLDER = "..\\dataset\\"

TEST_SUBFOLDER = os.path.join(MAIN_DATASET_FOLDER, "test")
TRAIN_SUBFOLDER = os.path.join(MAIN_DATASET_FOLDER, "train")

TEST_ESPAGNE_FOLDER = os.path.join(TEST_SUBFOLDER, "espagne")
TEST_FRANCE_FOLDER = os.path.join(TEST_SUBFOLDER, "france")
TEST_JAPON_FOLDER = os.path.join(TEST_SUBFOLDER, "japon")

TRAIN_ESPAGNE_FOLDER = os.path.join(TRAIN_SUBFOLDER, "espagne")
TRAIN_FRANCE_FOLDER = os.path.join(TRAIN_SUBFOLDER, "france")
TRAIN_JAPON_FOLDER = os.path.join(TRAIN_SUBFOLDER, "japon")


In [4]:
def fill_x_and_y_with_images_and_labels(folder, classe, dataset):

    for file in os.listdir(folder):

        file_path = os.path.join(folder, file)
        image = Image.open(file_path)
        image = image.resize((64, 64))
        if image.mode in ("RGB", "P"): image = image.convert("RGBA")
        im_arr = np.array(image).flatten()
        im_arr = im_arr / 255.0

        completed_dataset = {}
        completed_dataset["value"] = im_arr
        completed_dataset["classe"] = classe

        dataset.append(completed_dataset.copy())

In [5]:
def import_dataset(folder, classe):

    dataset = []

    fill_x_and_y_with_images_and_labels(folder, classe, dataset)

    return dataset

In [6]:
x_test = []
y_test = []

x_train = []
y_train = []

class_espagne_expected_outputs = [1, 0, 0]
dataset_test_for_espagne = import_dataset(TEST_ESPAGNE_FOLDER, class_espagne_expected_outputs)
dataset_train_for_espagne = import_dataset(TRAIN_ESPAGNE_FOLDER, class_espagne_expected_outputs)

class_france_expected_outputs = [0, 1, 0]
dataset_test_for_france = import_dataset(TEST_FRANCE_FOLDER, class_france_expected_outputs)
dataset_train_for_france = import_dataset(TRAIN_FRANCE_FOLDER, class_france_expected_outputs)

class_japon_expected_outputs = [0, 0, 1]
dataset_test_for_japon = import_dataset(TEST_JAPON_FOLDER, class_japon_expected_outputs)
dataset_train_for_japon = import_dataset(TRAIN_JAPON_FOLDER, class_japon_expected_outputs)

final_dataset_test = dataset_test_for_espagne + dataset_test_for_france + dataset_test_for_japon
final_dataset_train = dataset_train_for_espagne + dataset_train_for_france + dataset_train_for_japon

In [7]:
for value in final_dataset_test:

    the_image = value["value"].tolist()
    the_classe = value["classe"]

    x_test.append(the_image)
    y_test.append(the_classe)

for value in final_dataset_train:

    the_image = value["value"].tolist()
    the_classe = value["classe"]

    x_train.append(the_image)
    y_train.append(the_classe)

dataset_inputs = x_train
dataset_outputs = y_train

initiate_dataset_inputs_size = len(dataset_inputs[0])
initiate_dataset_inputs_type = ctypes.c_float * initiate_dataset_inputs_size

classes_initiate_dataset_inputs_size = initiate_dataset_inputs_size * 2

In [8]:
inputs_result = []

for sublist in dataset_inputs:
    for item in sublist:
        inputs_result.append(item)

dataset_inputs = inputs_result

outputs_result = []

for sublist in dataset_outputs:
    for item in sublist:
        outputs_result.append(item)

dataset_outputs = outputs_result

arr = [initiate_dataset_inputs_size, classes_initiate_dataset_inputs_size, 3]
arr_size = len(arr)
arr_type = ctypes.c_int * arr_size

In [9]:
# Convert train/test to array
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [10]:
#### ------------------------------------------- ####
#### ------------LOAD MODELS---------------------#### 
#### ------------------------------------------- ####

In [11]:
model_pmc_a = tf.keras.models.load_model("../models/keras_models/model_a.h5")

In [12]:
model_pmc_b = tf.keras.models.load_model("../models/keras_models/model_b.h5")

In [13]:
model_pmc_c = tf.keras.models.load_model("../models/keras_models/model_c.h5")

In [14]:
model_pmc_d = tf.keras.models.load_model("../models/keras_models/model_d.h5")

In [15]:
#### ------------------------------------------- ####
#### ------ DEFINE THE TRAINING ALGORITHM ------ #### 
#### ------------------------------------------- ####

In [16]:
OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=1e-3)
LOSS = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
METRICS = [
    tf.keras.metrics.CategoricalAccuracy(name='acc'),
]

In [17]:
model_pmc_a.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

model_pmc_b.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

model_pmc_c.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

model_pmc_d.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

In [18]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (A) ------------- #### 
#### ------------------------------------------- ####

In [19]:
tf.keras.backend.clear_session()

In [20]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_a.h5', save_best_only=True)

In [21]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_a_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_a = model_pmc_a.fit(
    x=x_train, 
    y=y_train, 
    epochs=100, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
        
        
    ]

)

Epoch 1/100
19/19 [==============================] - 1s 19ms/step - loss: 0.2947 - acc: 0.9067 - val_loss: 0.5715 - val_acc: 0.7167
Epoch 2/100
19/19 [==============================] - 0s 5ms/step - loss: 0.2899 - acc: 0.9100 - val_loss: 0.5800 - val_acc: 0.7083
Epoch 3/100
19/19 [==============================] - 0s 5ms/step - loss: 0.2860 - acc: 0.9100 - val_loss: 0.5444 - val_acc: 0.7833
Epoch 4/100
19/19 [==============================] - 0s 5ms/step - loss: 0.2886 - acc: 0.8900 - val_loss: 0.5614 - val_acc: 0.7083
Epoch 5/100
19/19 [==============================] - 0s 5ms/step - loss: 0.2795 - acc: 0.9067 - val_loss: 0.6403 - val_acc: 0.7417
Epoch 6/100
19/19 [==============================] - 0s 6ms/step - loss: 0.2859 - acc: 0.9033 - val_loss: 0.5337 - val_acc: 0.7333
Epoch 7/100
19/19 [==============================] - 0s 5ms/step - loss: 0.2711 - acc: 0.9167 - val_loss: 0.5754 - val_acc: 0.7083
Epoch 8/100
19/19 [==============================] - 0s 5ms/step - loss: 0.2820 - 

19/19 [==============================] - 0s 7ms/step - loss: 0.2307 - acc: 0.9250 - val_loss: 0.6094 - val_acc: 0.6917
Epoch 64/100
19/19 [==============================] - 0s 7ms/step - loss: 0.2374 - acc: 0.9367 - val_loss: 0.5838 - val_acc: 0.7167
Epoch 65/100
19/19 [==============================] - 0s 6ms/step - loss: 0.2233 - acc: 0.9400 - val_loss: 0.5769 - val_acc: 0.7250
Epoch 66/100
19/19 [==============================] - 0s 7ms/step - loss: 0.2369 - acc: 0.9283 - val_loss: 0.5848 - val_acc: 0.7000
Epoch 67/100
19/19 [==============================] - 0s 7ms/step - loss: 0.2375 - acc: 0.9317 - val_loss: 0.5501 - val_acc: 0.7333
Epoch 68/100
19/19 [==============================] - 0s 7ms/step - loss: 0.2359 - acc: 0.9300 - val_loss: 0.5557 - val_acc: 0.7500
Epoch 69/100
19/19 [==============================] - 0s 7ms/step - loss: 0.2508 - acc: 0.9233 - val_loss: 0.5397 - val_acc: 0.7417
Epoch 70/100
19/19 [==============================] - 0s 7ms/step - loss: 0.2407 - acc: 0

In [22]:
history_a.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [23]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (B) ------------- #### 
#### ------------------------------------------- ####

In [24]:
tf.keras.backend.clear_session()

In [25]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_b.h5', save_best_only=True)

In [26]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_b_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_b = model_pmc_b.fit(
    x=x_train, 
    y=y_train, 
    epochs=100, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
    ]
)

Epoch 1/100
19/19 [==============================] - 2s 74ms/step - loss: 0.4269 - acc: 0.8111 - val_loss: 0.6293 - val_acc: 0.7167
Epoch 2/100
19/19 [==============================] - 0s 5ms/step - loss: 0.4191 - acc: 0.8283 - val_loss: 0.5436 - val_acc: 0.7333
Epoch 3/100
19/19 [==============================] - 0s 5ms/step - loss: 0.4325 - acc: 0.8233 - val_loss: 0.5476 - val_acc: 0.7667
Epoch 4/100
19/19 [==============================] - 0s 6ms/step - loss: 0.4145 - acc: 0.8483 - val_loss: 0.6090 - val_acc: 0.7167
Epoch 5/100
19/19 [==============================] - 0s 7ms/step - loss: 0.3998 - acc: 0.8517 - val_loss: 0.6113 - val_acc: 0.7167
Epoch 6/100
19/19 [==============================] - 0s 7ms/step - loss: 0.4233 - acc: 0.8367 - val_loss: 0.6223 - val_acc: 0.7167
Epoch 7/100
19/19 [==============================] - 0s 6ms/step - loss: 0.4114 - acc: 0.8383 - val_loss: 0.6350 - val_acc: 0.7167
Epoch 8/100
19/19 [==============================] - 0s 5ms/step - loss: 0.4123 - 

19/19 [==============================] - 0s 9ms/step - loss: 0.3326 - acc: 0.8783 - val_loss: 0.5396 - val_acc: 0.7333
Epoch 64/100
19/19 [==============================] - 0s 5ms/step - loss: 0.3406 - acc: 0.8700 - val_loss: 0.5336 - val_acc: 0.7417
Epoch 65/100
19/19 [==============================] - 0s 7ms/step - loss: 0.3175 - acc: 0.8867 - val_loss: 0.6119 - val_acc: 0.7250
Epoch 66/100
19/19 [==============================] - 0s 6ms/step - loss: 0.3246 - acc: 0.8850 - val_loss: 0.5431 - val_acc: 0.7500
Epoch 67/100
19/19 [==============================] - 0s 7ms/step - loss: 0.3780 - acc: 0.8333 - val_loss: 0.7224 - val_acc: 0.7000
Epoch 68/100
19/19 [==============================] - 0s 7ms/step - loss: 0.3269 - acc: 0.8650 - val_loss: 0.5397 - val_acc: 0.7333
Epoch 69/100
19/19 [==============================] - 0s 6ms/step - loss: 0.3148 - acc: 0.9017 - val_loss: 0.5809 - val_acc: 0.7250
Epoch 70/100
19/19 [==============================] - 0s 4ms/step - loss: 0.3505 - acc: 0

In [27]:
history_b.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [28]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (C) ------------- #### 
#### ------------------------------------------- ####

In [29]:
tf.keras.backend.clear_session()

In [30]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_c.h5', save_best_only=True)

In [31]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_c_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_c = model_pmc_c.fit(
    x=x_train, 
    y=y_train, 
    epochs=100, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
    ]
)

Epoch 1/100
19/19 [==============================] - 1s 70ms/step - loss: 0.3508 - acc: 0.8417 - val_loss: 0.6288 - val_acc: 0.7333
Epoch 2/100
19/19 [==============================] - 0s 7ms/step - loss: 0.3358 - acc: 0.8800 - val_loss: 0.5464 - val_acc: 0.7500
Epoch 3/100
19/19 [==============================] - 0s 7ms/step - loss: 0.3343 - acc: 0.8883 - val_loss: 0.5734 - val_acc: 0.7500
Epoch 4/100
19/19 [==============================] - 0s 7ms/step - loss: 0.3447 - acc: 0.8767 - val_loss: 0.5502 - val_acc: 0.7417
Epoch 5/100
19/19 [==============================] - 0s 7ms/step - loss: 0.3392 - acc: 0.8750 - val_loss: 0.4981 - val_acc: 0.7833
Epoch 6/100
19/19 [==============================] - 0s 7ms/step - loss: 0.3444 - acc: 0.8617 - val_loss: 0.5287 - val_acc: 0.7333
Epoch 7/100
19/19 [==============================] - 0s 7ms/step - loss: 0.3239 - acc: 0.8783 - val_loss: 0.6082 - val_acc: 0.7083
Epoch 8/100
19/19 [==============================] - 0s 8ms/step - loss: 0.3298 - 

19/19 [==============================] - 0s 6ms/step - loss: 0.2556 - acc: 0.9183 - val_loss: 0.5011 - val_acc: 0.7917
Epoch 64/100
19/19 [==============================] - 0s 7ms/step - loss: 0.2535 - acc: 0.9233 - val_loss: 0.5977 - val_acc: 0.7333
Epoch 65/100
19/19 [==============================] - 0s 5ms/step - loss: 0.2678 - acc: 0.9033 - val_loss: 0.6323 - val_acc: 0.7667
Epoch 66/100
19/19 [==============================] - 0s 6ms/step - loss: 0.2626 - acc: 0.9100 - val_loss: 0.4908 - val_acc: 0.7750
Epoch 67/100
19/19 [==============================] - 0s 5ms/step - loss: 0.2585 - acc: 0.9083 - val_loss: 0.5857 - val_acc: 0.7750
Epoch 68/100
19/19 [==============================] - 0s 7ms/step - loss: 0.2484 - acc: 0.9167 - val_loss: 0.5472 - val_acc: 0.7750
Epoch 69/100
19/19 [==============================] - 0s 6ms/step - loss: 0.2353 - acc: 0.9317 - val_loss: 0.4944 - val_acc: 0.7667
Epoch 70/100
19/19 [==============================] - 0s 6ms/step - loss: 0.2777 - acc: 0

In [32]:
history_c.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [33]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (D) ------------- #### 
#### ------------------------------------------- ####

In [34]:
tf.keras.backend.clear_session()

In [35]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_d.h5', save_best_only=True)

In [36]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_d_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_d = model_pmc_d.fit(
    x=x_train, 
    y=y_train, 
    epochs=100, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
    ]
)

Epoch 1/100
19/19 [==============================] - 1s 71ms/step - loss: 0.3007 - acc: 0.8667 - val_loss: 0.5364 - val_acc: 0.7500
Epoch 2/100
19/19 [==============================] - 0s 7ms/step - loss: 0.2760 - acc: 0.9100 - val_loss: 0.5562 - val_acc: 0.7333
Epoch 3/100
19/19 [==============================] - 0s 8ms/step - loss: 0.2648 - acc: 0.9217 - val_loss: 0.6453 - val_acc: 0.7167
Epoch 4/100
19/19 [==============================] - 0s 8ms/step - loss: 0.2663 - acc: 0.9117 - val_loss: 0.5862 - val_acc: 0.7250
Epoch 5/100
19/19 [==============================] - 0s 6ms/step - loss: 0.2839 - acc: 0.8917 - val_loss: 0.5348 - val_acc: 0.7083
Epoch 6/100
19/19 [==============================] - 0s 10ms/step - loss: 0.2924 - acc: 0.8817 - val_loss: 0.6844 - val_acc: 0.7167
Epoch 7/100
19/19 [==============================] - 0s 8ms/step - loss: 0.3049 - acc: 0.8867 - val_loss: 0.7940 - val_acc: 0.7083
Epoch 8/100
19/19 [==============================] - 0s 7ms/step - loss: 0.2689 -

19/19 [==============================] - 0s 5ms/step - loss: 0.2301 - acc: 0.9133 - val_loss: 0.6032 - val_acc: 0.7333
Epoch 64/100
19/19 [==============================] - 0s 5ms/step - loss: 0.2117 - acc: 0.9333 - val_loss: 0.5478 - val_acc: 0.7083
Epoch 65/100
19/19 [==============================] - 0s 6ms/step - loss: 0.2663 - acc: 0.9167 - val_loss: 0.6621 - val_acc: 0.7250
Epoch 66/100
19/19 [==============================] - 0s 6ms/step - loss: 0.2306 - acc: 0.9200 - val_loss: 0.5785 - val_acc: 0.7167
Epoch 67/100
19/19 [==============================] - 0s 6ms/step - loss: 0.2065 - acc: 0.9400 - val_loss: 0.5855 - val_acc: 0.7417
Epoch 68/100
19/19 [==============================] - 0s 6ms/step - loss: 0.2209 - acc: 0.9283 - val_loss: 0.6432 - val_acc: 0.7417
Epoch 69/100
19/19 [==============================] - 0s 5ms/step - loss: 0.2350 - acc: 0.9200 - val_loss: 0.5493 - val_acc: 0.7500
Epoch 70/100
19/19 [==============================] - 0s 5ms/step - loss: 0.1966 - acc: 0

In [37]:
history_d.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [ ]:
#### ------------------------------------------- ####
#### ------------- SHOW CURVES ------------------#### 
#### ------------------------------------------- ####

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 11448), started 0:26:20 ago. (Use '!kill 11448' to kill it.)